<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Parse-the-data" data-toc-modified-id="Parse-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parse the data</a></span></li><li><span><a href="#Data-Exploration" data-toc-modified-id="Data-Exploration-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Exploration</a></span></li><li><span><a href="#Pre-processing-Functions" data-toc-modified-id="Pre-processing-Functions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Pre-processing Functions</a></span><ul class="toc-item"><li><span><a href="#Look-up-Table" data-toc-modified-id="Look-up-Table-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Look-up Table</a></span></li><li><span><a href="#Tokenize-Punctuation" data-toc-modified-id="Tokenize-Punctuation-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Tokenize Punctuation</a></span></li><li><span><a href="#Apply-the-pre-processing-functions" data-toc-modified-id="Apply-the-pre-processing-functions-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Apply the pre-processing functions</a></span></li></ul></li><li><span><a href="#Neural-Network" data-toc-modified-id="Neural-Network-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Neural Network</a></span><ul class="toc-item"><li><span><a href="#Input:-Batching" data-toc-modified-id="Input:-Batching-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Input: Batching</a></span></li><li><span><a href="#Neural-Network-Architecture" data-toc-modified-id="Neural-Network-Architecture-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Neural Network Architecture</a></span></li><li><span><a href="#Forward-and-Backpropagation" data-toc-modified-id="Forward-and-Backpropagation-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Forward and Backpropagation</a></span></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="#Training-Function" data-toc-modified-id="Training-Function-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Training Function</a></span></li><li><span><a href="#Hyperparameters" data-toc-modified-id="Hyperparameters-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Hyperparameters</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Train</a></span></li></ul></li><li><span><a href="#Generate-a-TV-Script" data-toc-modified-id="Generate-a-TV-Script-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Generate a TV Script</a></span><ul class="toc-item"><li><span><a href="#function-to-generate-text" data-toc-modified-id="function-to-generate-text-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>function to generate text</a></span></li><li><span><a href="#Apply-the-function-to-make-a-script" data-toc-modified-id="Apply-the-function-to-make-a-script-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Apply the function to make a script</a></span></li><li><span><a href="#Save-a-Script" data-toc-modified-id="Save-a-Script-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Save a Script</a></span></li></ul></li><li><span><a href="#Submitting-This-Project" data-toc-modified-id="Submitting-This-Project-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Submitting This Project</a></span></li></ul></div>

# Libraries

In [200]:
import pandas as pd
import numpy as np
from collections import Counter

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import pickle

# Data

In [2]:
data_dir = './data/seinfeld-chronicles/scripts.csv'

In [3]:
scripts_df = pd.read_csv(data_dir)
scripts_df.head()

,Unnamed: 0,Character,Dialogue,EpisodeNo,SEID,Season
0,0,JERRY,Do you know what this is all about? Do you kno...,1.0,S01E01,1.0
1,1,JERRY,"(pointing at Georges shirt) See, to me, that b...",1.0,S01E01,1.0
2,2,GEORGE,Are you through?,1.0,S01E01,1.0
3,3,JERRY,"You do of course try on, when you buy?",1.0,S01E01,1.0
4,4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",1.0,S01E01,1.0


# Parse the data
Desired format:
<br>
all one string, with the character name, colon, then lower-cased dialoge. 
<br>
Example
<br>
'jerry:  do you know what this is all about? do you know, why were here?'

In [4]:
# Function to format text
def scriptParser(character, dialogue):
    if(isinstance(dialogue, str)):
        dialogue = dialogue.lower()
    else:
        #dialogue = str(dialogue)
        #print(dialogue) - always a nan
        dialogue = ''
    if(isinstance(character, str)):
        character = character.lower()
    else:
        #character = str(character)
        character = ''
    
    return character+': '+dialogue+'\n\n'

In [5]:
# test the function on one row
row = 0
scriptParser(scripts_df['Character'].iloc[row], scripts_df['Dialogue'].iloc[row])

'jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people tryin to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, whatta you do? you go we gotta be getting back. once youre out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? where ever you are in life, its my feeling, youve gotta go.\n\n'

In [6]:
# test the function while joining on 5 rows
''.join([scriptParser(row['Character'], row['Dialogue']) 
         for index, row in scripts_df[:5].iterrows()])

'jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people tryin to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, whatta you do? you go we gotta be getting back. once youre out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? where ever you are in life, its my feeling, youve gotta go.\n\njerry: (pointi

In [182]:
# apply the function
text = ''.join([scriptParser(row['Character'], row['Dialogue']) 
         for index, row in scripts_df.iterrows()])

# Data Exploration

In [8]:
view_line_range = (0, 10)

print('Dataset Stats')
print('~ Number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
~ Number of unique words: 46380
Number of lines: 109233
Average number of words in each line: 5.544121282030155

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people tryin to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, whatta you do? you go we gotta be getting back. once youre out, you wanna get back! you wanna go 

# Pre-processing Functions

## Look-up Table
2 dictionaries:
<br>
1. word to index: words2idx 
2. index to word: idx2word 

More common words should have a lower index

In [9]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    
    text: The text of tv scripts split into words
    returns: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # first get the word_counts
    word_counts = Counter(text)
    
    # sort from most to least frequent
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    
    # define the dictionaries
    idx2word = {ii: word for ii, word in enumerate(sorted_vocab)}
    word2idx = {word: ii for ii, word in idx2word.items()}
    
    # return tuple
    return (idx2word, word2idx)


## Tokenize Punctuation

In [10]:
# create a dict to turn punctuation into a token.
punct2token = {'.': '<PERIOD>',
                ',': '<COMMA>',
                '"': '<QUOTATION_MARK>',
                ';': '<SEMICOLON>',
                '!': '<EXCLAMATION_MARK>',
                '?': '<QUESTION_MARK>',
                '(': '<LEFT_PAREN>',
                ')': '<RIGHT_PAREN>',
                '--': ' <HYPHENS> ',
                '-': '<DASH>',
                '?': '<QUESTION_MARK>',
                '\n': '<NEW_LINE>',
                ':': ' <COLON> '}


In [11]:
punct2token[',']

'<COMMA>'

## Apply the pre-processing functions

In [183]:
# save the original text 
raw_text = text

In [12]:
# Tokenize the punctuation
for punct, token in punct2token.items():
    text = text.replace(punct, ' {} '.format(token))

In [13]:
# split and make all ensure all text is lower case
text = text.lower()
text = text.split()

In [14]:
# create the vocab dictionaries
idx2word, word2idx = create_lookup_tables(text + ['<PAD>'])

In [15]:
# apply dictionaries to text
int_text = [word2idx[word] for word in text]

In [142]:
int_text

[8,
 2,
 35,
 5,
 27,
 19,
 24,
 23,
 49,
 58,
 4,
 35,
 5,
 27,
 3,
 80,
 119,
 61,
 4,
 9,
 54,
 47,
 3,
 24,
 23,
 47,
 1,
 1,
 1,
 18,
 47,
 23,
 79,
 21,
 7,
 1279,
 547,
 8340,
 6825,
 21,
 240,
 1,
 147,
 1,
 1,
 1,
 81,
 5,
 198,
 235,
 147,
 206,
 58,
 55,
 133,
 63,
 47,
 4,
 24,
 23,
 19,
 692,
 206,
 58,
 1,
 1,
 1,
 24,
 218,
 125,
 3,
 119,
 49,
 47,
 83,
 3,
 26,
 79,
 23,
 288,
 1,
 45,
 79,
 375,
 61,
 23,
 288,
 3,
 119,
 49,
 47,
 12,
 75,
 48,
 147,
 6826,
 9,
 247,
 191,
 3,
 64,
 202,
 27,
 127,
 55,
 48,
 1,
 13,
 29,
 95,
 2681,
 171,
 14,
 81,
 5,
 1403,
 4,
 3,
 6,
 388,
 247,
 68,
 1,
 127,
 81,
 36,
 63,
 4,
 36,
 530,
 94,
 25,
 127,
 36,
 42,
 78,
 1,
 36,
 337,
 40,
 577,
 47,
 1,
 5,
 213,
 63,
 47,
 5,
 51,
 426,
 3,
 5,
 323,
 47,
 7,
 561,
 3,
 57,
 4,
 5,
 100,
 7,
 732,
 3,
 5,
 51,
 49,
 426,
 3,
 51,
 7,
 1253,
 3,
 51,
 52,
 353,
 3,
 7,
 158,
 3,
 7,
 834,
 3,
 7,
 2954,
 1,
 1,
 1,
 114,
 313,
 674,
 173,
 3,
 4640,
 5,
 35,
 4,
 5,
 63,
 55,
 

In [16]:
# save to a pickle file
pickle.dump((int_text, idx2word, word2idx, punct2token), open('preprocess.p', 'wb'))

# Neural Network

In [17]:
# Check GPU Access
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

No GPU found. Please use a GPU to train your neural network.


## Input: Batching
We want to batch the data based on a provided sequence legnth
Example Input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

`features` Tensor:
```
[1, 2, 3, 4]
```
`target` Tensor (the next "word"):
```
5
```

Next `features` and `target` Tensors:
```
[2, 3, 4, 5]  # features
6             # target

In [167]:
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    param words: The word ids of the TV scripts
    param sequence_length: The sequence length of each batch
    param batch_size: The size of each batch; the number of sequences in a batch
    return: DataLoader with batched data
    """
    
    # iterate overall all words
    features = []
    targets = []
    for i, w in enumerate(words):
        if((i+sequence_length) < len(words)):
            features.append(words[i:(i+sequence_length)])
            targets.append(words[i+sequence_length])
    
    features = np.array(features)
    targets = np.array(targets)
    
    # convert numpy arrays into a torch tensor dataset
    sequenced_data = TensorDataset(torch.from_numpy(features), torch.from_numpy(targets))
    
    # create the final batch DataLoader
    batch_dL = DataLoader(sequenced_data, shuffle=True, batch_size=batch_size) 

    # return a dataloader
    return batch_dL



In [168]:
# Test the Data Loader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[42, 43, 44, 45, 46],
        [ 8,  9, 10, 11, 12],
        [10, 11, 12, 13, 14],
        [28, 29, 30, 31, 32],
        [41, 42, 43, 44, 45],
        [24, 25, 26, 27, 28],
        [ 9, 10, 11, 12, 13],
        [44, 45, 46, 47, 48],
        [35, 36, 37, 38, 39],
        [20, 21, 22, 23, 24]], dtype=torch.int32)

torch.Size([10])
tensor([47, 13, 15, 33, 46, 29, 14, 49, 40, 25], dtype=torch.int32)


## Neural Network Architecture
Define an LSTM-based RNN class

In [170]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
       
         # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.drop_prob = dropout
        self.batch_size = None
        
       
        # define model layers
        
        # embedding layer 
        self.embed = nn.Embedding(self.vocab_size, self.embedding_dim)
        
        # LSTM layer
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, self.n_layers, dropout=self.drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(self.drop_prob)
        
        # fully-connected layer
        self.fc = nn.Linear(self.hidden_dim, self.output_size)
        
        # initialize some weights in the network 
        self.init_weights()
       
    
    def init_weights(self):
        i_range = 0.08
        
        self.embed.weight.data.uniform_(-i_range, i_range)
        
        self.lstm.weight_ih_l0.data.uniform_(-i_range, i_range)
        self.lstm.weight_hh_l0.data.uniform_(-i_range, i_range)
        
        self.lstm.bias_ih_l0.data.zero_()
        self.lstm.bias_hh_l0.data.zero_()
        
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.normal_(0.0, (1.0 /np.sqrt(self.fc.in_features)))
        
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # embeddings
        x = self.embed(nn_input)
        
        # get the LSTM outputs
        l_out, hidden = self.lstm(x, hidden)
        
        # stack up the LSTM outputs - aka shape the output
        l_out = l_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout - doesn't seem to be helping
        # out = self.dropout(l_out)
        
        # get the final output
        out = self.fc(l_out)

        # reshape into (batch_size, seq_length, output_size)
        out = out.view(self.batch_size, -1, self.output_size)

        # get last batch
        out = out[:, -1]
        
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        # and move to GPU if available
        
        weight = next(self.parameters()).data
        
        self.batch_size = batch_size
        
        if(train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                     weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                     weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden


## Forward and Backpropagation
Define a function 
```
forward_back_prop()
```
That will be used in training to find the average loss over a batch of data.

In [195]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    param decoder: The PyTorch Module that holds the neural network
    param decoder_optimizer: The PyTorch optimizer for the neural network
    param criterion: The PyTorch loss function
    param inp: A batch of input to the neural network
    param target: The target output for the batch of input
    return: The loss and the latest hidden state Tensor
    """
    
    # move data to GPU, if available
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
    else:
        inp, target = inp.long(), target.long()
        
    # create new varaibles for the hidden state
    hidden = tuple([each.data for each in hidden])
    
    # zero out any accumulated gradients
    rnn.zero_grad()
    
    # get the model outputs
    output, hidden = rnn(inp, hidden)
    
    # calculate the loss & perform backprop
    loss = criterion(output.squeeze(), target)
    loss.backward()
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    clip = 5
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    
    # perform an optimization step
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Training

## Training Function

In [173]:
# function to train the rnn
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()
    
    # initialize a high minimum loss
    min_Loss = np.Inf

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                avg_Loss = np.average(batch_losses)
                
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, avg_Loss))
                
                if avg_Loss < min_Loss:
                    min_Loss = avg_Loss
                    helper.save_model('./save/trained_rnn_new', rnn)
                    print('Model Trained and Saved')
                    
                batch_losses = []

    # returns a trained rnn
    return rnn

## Hyperparameters

In [184]:
# Find the average sentence length to use for the sequence length
temp = raw_text.replace('?', '.')
temp = temp.replace('!', '.')
sentences = temp.split('.')

word_count_sentences = [len(sent.split()) for sent in sentences]
print('Average number of words in each sentence: {}'.format(np.average(word_count_sentences)))

Average number of words in each sentence: 5.756923863539527


In [205]:
# Data params

# Sequence Length
sequence_length = 6  # of words in a sequence

# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [206]:
# Training parameters

# Number of Epochs
num_epochs = 1 # go low on CPU - requires 10 to get below 3.5

# Learning Rate
learning_rate = 0.002


In [207]:
# Model parameters

# Vocab size
vocab_size = len(word2idx)

# Output size
output_size = vocab_size

# Embedding Dimension
embedding_dim = 256

# Hidden Dimension
hidden_dim = 256

# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 1000

## Train

In [208]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

print('done!')

Training for 1 epoch(s)...
Epoch:    1/1     Loss: 5.48744264125824

Epoch:    1/1     Loss: 5.0490784506797795

Epoch:    1/1     Loss: 4.9951553716659545

Epoch:    1/1     Loss: 4.996782721996308

Epoch:    1/1     Loss: 4.972678839683533

Epoch:    1/1     Loss: 4.9234851174354555

Epoch:    1/1     Loss: 4.947990944385529

Epoch:    1/1     Loss: 4.866194286346436

Epoch:    1/1     Loss: 4.820928164482117

Epoch:    1/1     Loss: 4.7875993790626525

Epoch:    1/1     Loss: 4.828715405464172

Epoch:    1/1     Loss: 4.804311979293823

Epoch:    1/1     Loss: 4.818021938800812

Epoch:    1/1     Loss: 4.731715711593628

Epoch:    1/1     Loss: 4.747332892894745

Epoch:    1/1     Loss: 4.727448909282685

Epoch:    1/1     Loss: 4.698231386184692

Epoch:    1/1     Loss: 4.773421789169311

Epoch:    1/1     Loss: 4.697781580448151

Epoch:    1/1     Loss: 4.716354287147522

Epoch:    1/1     Loss: 4.679349272727967

Epoch:    1/1     Loss: 4.741835752010346

Epoch:    1/1     Loss: 

KeyboardInterrupt: 

In [ ]:
# saving the trained model
# save_filename = os.path.splitext(os.path.basename('./save/trained_rnn'))[0] + '.pt'
# torch.save(trained_rnn, save_filename)
# print('Model Trained and Saved')

# Generate a TV Script
To create text, provide the network with a single 'seed' word, and it will keep making predictions until it reaches a set length

In [ ]:
# loading a trained model
filename = 'trained_rnn_new'
save_filename = os.path.splitext(os.path.basename(filename))[0] + ' .pt'
# trained_rnn = troch.load(save_filename)

## function to generate text

In [ ]:
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences


## Apply the function to make a script
start with a character name:
- jerry
- elaine
- george
- kramer

In [ ]:
gen_length = 400 
prime_word = 'jerry'


pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

## Save a Script

In [ ]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()


# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.